In [12]:
from sage.all import CyclotomicField
from decimal import Decimal
from matplotlib import colormaps
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors
import threading
import stopit
import os

def subst_dictionary(template, dictionary):
        for ky, val in dictionary.items():
            if type(val) == list: # Remove the square brackets if val is a Python list.
               template = template.replace(f'{{{ky}}}', str(latex(val))[6:-7])  # latex(str(val)[1:-1])
            if ky[-3:] == "alt":
               template = template.replace(f'{{{ky}}}', val)
            else:
                template = template.replace(f'{{{ky}}}', latex(val))
        return template

def coprimes(k):
    return [i for i in range(1, k) if gcd(i, k) == 1]
        
def coset_reps(coprimes_list, l, k):
    list_aux = coprimes_list.copy()
    for i in range(len(list_aux)):
        list_aux.remove(list_aux[i]*l % k)
        if i == len(list_aux)-1:
            return list_aux

def try_reps_list(coset_reps_list, prime_example, l, k):
    return [m for m in coset_reps_list if l*m % k == prime_example]

def f_polynomial_roots(l, k, u, coset_reps_list):
    z = CyclotomicField(k, names = 'ζ').gen()
    return [h(l,k,u)(z**m) for m in coset_reps_list]

def h(l,k,u):
    z = QQ['x'].gen()
    return -(u - z)*(u - z**l)
    
def polynomial(root_list):
    R = QQ['x']
    x = R.gen()
    return R(prod(x-root for root in root_list))

def find_prime(l, k, discf, max_iter = 10000):
    i = 0
    iter_count = 0
    
    if discf != 0:
        while not (ZZ(i*k+l).is_prime() and discf % (i*k+l) != 0):
            i += 1
            iter_count += 1
            if iter_count > max_iter:
                raise ValueError(f"No suitable prime found within {max_iter} iterations.")
    else:
        while not ZZ(i*k+l).is_prime():
            i += 1
            iter_count += 1
            if iter_count > max_iter:
                raise ValueError(f"No suitable prime found within {max_iter} iterations.")
    return i*k+l

def prev_primes(prime, l, k):
    return sorted([prime-i for i in range(0, prime, 1) if ((prime-i) % k == l and ZZ(prime-i).is_prime())])

def prime_divisors(factored_num):
    return [p for p, e in factored_num]

def cyclotomic_roots(k):
    z = CyclotomicField(k, names = 'ζ').gen()
    return [z ** i for i in range(1, k + 1) if gcd(i, k) == 1]

def find_b_value(f, prime, max_iter = 10000):
    b = 0
    while b < max_iter:
        f_b = f(x=b) 
        if f_b % prime == 0 and f_b % (prime ** 2) != 0:
            return b
        b += 1
    raise ValueError(f"Could not find a valid b value within {max_iter} iterations.")

def dividend_check(l,k,u,p):
    x = QQ['x'].gen()
    return -u*(x^p+x^(l*p))+x^((1+l)*p)+u*(x+x^l)-x^(1+l)

def sufix_cyclo(k):
    S = [4,5,6,7,8,9,0]
    if k == 11 or k == 12 or k == 13 or k % 10 in S:
        return 'th'
    elif k % 10 == 1:
        return 'st'
    elif k % 10 == 2:
        return 'nd'
    else:
        return 'rd'

def ell_options(k):
    if not k.is_integer() or k <= 0:
        raise ValueError("k must be a positive integer.")
    return [m for m in range(1,k) if m^2%k == 1]

###### Functions to test the limit of the program ######

def factor_timeout(n, timeout = 2):
    with stopit.ThreadingTimeout(timeout) as to_ctx_mgr: 
        ans = fork(lambda n: ZZ(n).factor(), timeout = timeout)(n)
        return ans
    if to_ctx_mgr.state in [to_ctx_mgr.TIMED_OUT, to_ctx_mgr.INTERRUPTED]:
        print('The factorization takes too long.')
        raise RuntimeError

def exec_times(first_k, last_k):
    os.environ["PATH"] = "/Library/TeX/texbin:" + os.environ["PATH"]
    plt.rcParams['text.usetex'] = True
    plt.rcParams['font.family'] = 'serif'
    
    hor_size = last_k-first_k+1
    ver_size = max([len(ell_options(QQ(k))) for k in range (first_k, last_k+1)])
    
    matrix = [[10 for _ in range(first_k, last_k+1)] for _ in range(1,last_k)]
    euclid_cases = [(k,l) for k in range(first_k, last_k+1) for l in range(1,k) if gcd(l,k) == 1 and l*l % k == 1]

    prev_k = 0
    i = 0
    for item in euclid_cases:
        if item[0] != prev_k:
            i = 0 
        try:
            t = cputime()
            ap_euc(ZZ(item[0]), ZZ(item[1]))
            matrix[i][item[0]-first_k] = cputime(t)
        except RuntimeError:
            matrix[i][item[0]-first_k] = 10
        i += 1
        prev_k = item[0]

    n = np.arange(hor_size * ver_size, dtype=float).reshape((ver_size, hor_size))
    for j in range(0, ver_size):
        for i in range(0, hor_size):
            n[j][i] = matrix[j][i]
    
    # Limits for the extent
    x_start = first_k - 0.5
    x_end = last_k + 0.5
    y_start = 0.5
    y_end = ver_size + 0.5 
    
    extent = [x_start, x_end, y_start, y_end]
    
    # The normal figure
    fig = plt.figure(figsize=(16, 12))
    ax = fig.add_subplot(111)
    masked_array = np.ma.masked_where(n == 10, n)
    cmap = matplotlib.cm.RdYlGn_r # Can be any colormap that you want after the cm

    cmap.set_bad(color = 'white')
    im = ax.imshow(masked_array, extent=extent, origin='lower', cmap=cmap)

    for x_index in range(first_k, last_k+1):
        for y_index in range(0, len(ell_options(QQ(x_index)))):
            text_x = x_index
            text_y = y_index + 1
            if n[y_index][x_index-first_k] == 10:
                label = ell_options(QQ(x_index))[y_index]
                ax.text(text_x, text_y, label, color = 'red', fontweight = 'bold', ha = 'center', va = 'center')
            else:
                label = ell_options(QQ(x_index))[y_index]
                ax.text(text_x, text_y, label, color = 'black', ha = 'center', va = 'center')
            
    x_label_list = [i for i in range(first_k, last_k+1)]
    y_label_list = [i for i in range(1, ver_size+1)]
    ax.set_xticks([i for i in range(first_k, last_k+1)])
    ax.set_yticks([i for i in range(1, ver_size+1)])
    ax.tick_params()
    ax.set_xticklabels(x_label_list)
    ax.set_yticklabels(y_label_list)
    plt.xlabel(r"$k$", fontsize = '14')
    plt.ylabel("Number of Euclidean proofs", fontsize = '14')
    cbar = fig.colorbar(im, orientation = "horizontal", pad = 0.07)
    cbar.set_label('Execution time [s]', fontsize = '12')
    plt.savefig("running_times.eps", format = "eps")
    #plt.savefig('running_times.png')
    plt.show()
    return

###### Functions to show the extent of the program ######

def extent_code(first_k, last_k):
    #plt.rcParams.update({
    #"text.usetex": True,
    #"font.family": "Computer Modern Roman"
    #}) # Only for CoCalc
    
    list_ell = [len(ell_options(QQ(k))) for k in range (first_k, last_k+1)]
    ratio = np.zeros(last_k-first_k+1) 
    for i in range(0,last_k-first_k+1):
        ratio[i] = list_ell[i]/euler_phi(i+first_k) 
    
    fig = plt.figure(figsize = (13, 9))
    ax = fig.add_subplot(111)
    plt.bar(range(first_k,last_k+1), 100*ratio, color = 'royalblue', width = 0.4)
    x_label_list = [i for i in range(first_k, last_k+1)]
    y_label_list = [i for i in range(0, 110, 10)]
    ax.set_xticks([i for i in range(first_k, last_k+1)])
    ax.set_yticks([i for i in range(0, 110, 10)])
    ax.tick_params()
    ax.set_xticklabels(x_label_list)
    ax.set_yticklabels(y_label_list)
    plt.xlabel(r"$k$", fontsize = '15')
    plt.ylabel(r"$\%$ of residue classes covered", fontsize = '15')
    #plt.savefig('Percentage_of_Euc.eps', format = 'eps')
    plt.savefig('Percentage_of_Euc.png')
    plt.show()